# Playground with generated data

## Dependencies

In [ ]:
:dep plotly = { version = ">=0.6.0" }
:dep sense-motive  = { path = ".." }

In [ ]:
extern crate plotly;
extern crate rand;
extern crate sense_motive;

In [ ]:
use std::f64::consts::PI;

use plotly::{
    color::{NamedColor}
    common::{Line, Marker, MarkerSymbol, Mode},
    Plot, Scatter, Trace
};
use rand::{
    distributions::{uniform::SampleUniform, Uniform},
    Rng,
};
use sense_motive::{Model, ModelBuilder, Transformation};

## Configuration

Model parameters:

In [ ]:
const N: usize = 1024;  // No. of original samples
const M: usize = 256;   // No. of compressed samples
const K: usize = 32;   // No. of sparse components

const TRANSFORM: Transformation = Transformation::Dct1dInverse; // Transformation used

Plot setting:

In [ ]:
let default_marker = Marker::new().size(4);

## Simulation

Build the model:

In [ ]:
let model: Model = ModelBuilder::new()
    .with_transformation(TRANSFORM)
    .build(M,N);

Generate the original signal and compress it :

In [ ]:
let original = match TRANSFORM {
    Transformation::None => generate_sparse_signal(K),
    Transformation::Dct1dInverse => generate_cos_signal(K),
    Transformation::Dct1d => generate_cos_signal(K),
    Transformation::Fourier1dInverse => todo!(),
    Transformation::Fourier1d => todo!(),
};

let mut plot = Plot::new();

let trace = Scatter::new((0..N - 1).collect(), original.clone())
    .name("Original")
    .mode(Mode::Lines);
plot.add_trace(trace);

plot.notebook_display();

fn generate_cos_signal(sparsity: usize) -> Vec<f64> {
    let frequencies = random_values(sparsity, 0.0, 10.0);
    let amplitudes = random_values(sparsity, 0.0, 1.0);
    let params: Vec<(f64, f64)> = frequencies.zip(amplitudes).collect();

    (0..N)
        .map(|i| i as f64 / N as f64)
        .map(|i| {
            params
                .iter()
                .map(|(f, a)| (a * (i * 2.0 * PI * f).cos()))
                .sum()
        })
        .collect()
}

fn generate_sparse_signal(sparsity: usize) -> Vec<f64> {
    let indices = random_values(sparsity, 0, N);
    let amplitudes = random_values(sparsity, 0.0, 1.0);

    let mut signal = vec![0.0; N];
    for (i, a) in indices.zip(amplitudes) {
        signal[i] = a;
    }

    signal
}

fn random_values<T>(n: usize, min: T, max: T) -> impl Iterator<Item = T>
where
    T: SampleUniform,
{
    rand::thread_rng()
        .sample_iter(Uniform::new(min, max))
        .take(n)
}



Compress it:

In [ ]:

let compressed = model.compress(&original);

let mut plot = Plot::new();

let trace = Scatter::new((0..N - 1).collect(), compressed.clone())
    .name("Compressed")
    .mode(Mode::LinesMarkers)
    .marker(default_marker.clone().symbol(MarkerSymbol::Cross));
plot.add_trace(trace);


plot.notebook_display();

Decompress:

In [ ]:
let decompressed = model.decompress(&compressed);
let error = difference(&original, &decompressed);


let mut plot = Plot::new();

let trace = Scatter::new((0..N - 1).collect(), original.clone())
    .name("Original")
    .mode(Mode::Lines);
plot.add_trace(trace);

let trace = Scatter::new((0..N - 1).collect(), decompressed.clone())
    .name("Decompressed")
    .mode(Mode::LinesMarkersText)
    .marker(default_marker.clone().symbol(MarkerSymbol::Circle));
plot.add_trace(trace);

let trace = Scatter::new((0..N - 1).collect(), error.clone())
    .name("Error")
    .mode(Mode::LinesText)
        .line(Line::new().color(NamedColor::Red).width(0.5));
plot.add_trace(trace);

plot.notebook_display();

fn difference(original: &Vec<f64>, decompressed: &Vec<f64>) -> Vec<f64> {
    original
        .iter()
        .zip(decompressed.iter())
        .map(|(a, b)| (a - b))
        .collect()
}

Total error:

In [ ]:
println!(
    "Error (normalized to N) {}",
    norm_l2(&error) / (N as f64)
);

fn norm_l2(vec: &Vec<f64>) -> f64 {
    vec.iter()
        .map(|i| i.powi(2))
        .sum()
}
